In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Dense

In [3]:
def ReadCSV(path):
    data = pd.read_csv(path)
    return data

In [4]:
#Load Silver Prices
Silver = ReadCSV('Silver_Price.csv')
Silver = Silver[['Date', 'Price']]
Silver['Date'] = pd.to_datetime(Silver['Date'])
Silver = Silver.reset_index()


#Load Crude Oil Prices
COil = ReadCSV('CrudeOil_Price.csv')
COil['Date'].apply(lambda x: datetime.datetime.strptime(x, '%m-%d-%Y') if type(x)==str else np.NaN)
COil['Date'] = pd.to_datetime(COil['Date'])
COil = COil.reset_index()

#Load Gold Prices (different method cause it's a xlsx file)
Gold = pd.read_excel('Gold_Price.xlsx', sheet_name = "Monthly_Average",usecols = "D,E")
#Resize Gold Prices to select the desired period of time
Gold = Gold.drop(labels=range(0, 93), axis=0)
Gold = Gold.drop(labels=range(501, 524), axis=0)
Gold.rename(columns = {'Unnamed: 3':'Date', 'Unnamed: 4':'Price','Unnamed: 5':'EUR', 'Unnamed: 6':'GBP', 'Unnamed: 7':'JPN'}, inplace = True)
Gold = Gold.loc[::-1]
Gold = Gold.reset_index()


#Load USD-EUR
USD_EUR = ReadCSV('USD-EUR.csv')
USD_EUR['Date'] = pd.to_datetime(USD_EUR['Date'])
USD_EUR = USD_EUR.reset_index()

#Load USD-CNY
USD_CNY = ReadCSV('USD-CNY.csv')
USD_CNY['Date'] = pd.to_datetime(USD_CNY['Date'])
USD_CNY = USD_CNY.loc[::-1]
USD_CNY = USD_CNY.reset_index()

#Load SP_500 data
SP_500 = ReadCSV('SP_500.csv')
SP_500 = SP_500.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis=1)
SP_500 = SP_500[['Date','Price']]
SP_500.astype({'Price': 'double'}).dtypes
SP_500['Date'] = pd.to_datetime(SP_500['Date'])


In [5]:
columns = [Silver["Price"], COil["Price"], SP_500["Price"],USD_EUR["Price"], USD_CNY["Price"]]

headers = ["Silver", "COil", "SP_500", "USD_EUR", "USD_CNY"]

data = pd.concat(columns, axis=1, keys=headers)

In [6]:
data

,Silver,COil,SP_500,USD_EUR,USD_CNY
0,18.404,59.88,3230.78,1.111345,7.0137
1,17.626,57.03,3140.98,1.105095,7.0199
2,18.699,53.96,3037.56,1.105257,7.0961
3,17.583,56.95,2976.74,1.100390,7.1137
4,18.877,54.81,2926.46,1.112623,7.0629
...,...,...,...,...,...
403,5.239,15.38,247.35,0.947997,3.2014
404,5.127,12.84,235.52,0.930534,3.2143
405,5.130,12.61,238.90,0.937445,3.2202
406,5.630,15.46,226.92,0.912133,3.2152


In [7]:
X = data
Y = Gold['Price']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=2)

# Random Forest

In [8]:
RF = RandomForestRegressor(100)
RF = RF.fit(X_train, Y_train);
scores = cross_val_score(RF, X=X_train, y=Y_train, cv=5, n_jobs=1)

In [9]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.98892256 0.98289389 0.98932041 0.99628867 0.99382975]


In [10]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.990 +/- 0.005


In [11]:
RF = RandomForestRegressor(100)
RF = RF.fit(X_train, Y_train);
scores = cross_val_score(RF, X=X_train, y=Y_train, cv=3, n_jobs=1)

In [12]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.98643104 0.97937162 0.99300082]


In [13]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.986 +/- 0.006


# CatBoost

In [14]:
CatB = CatBoostRegressor()
Y = Y_train.astype('int32')
CatB = CatB.fit(X_train,Y)
scores = cross_val_score(CatB, X=X_train, y=Y, cv=3, n_jobs=1)

Learning rate set to 0.034298
0:	learn: 445.3383297	total: 164ms	remaining: 2m 43s
1:	learn: 433.3776846	total: 166ms	remaining: 1m 23s
2:	learn: 420.9934949	total: 168ms	remaining: 55.9s
3:	learn: 408.9589394	total: 169ms	remaining: 42.2s
4:	learn: 398.3737284	total: 170ms	remaining: 33.9s
5:	learn: 387.0923653	total: 171ms	remaining: 28.4s
6:	learn: 376.2407186	total: 172ms	remaining: 24.4s
7:	learn: 365.7209279	total: 173ms	remaining: 21.5s
8:	learn: 355.4582600	total: 174ms	remaining: 19.1s
9:	learn: 345.7560879	total: 175ms	remaining: 17.3s
10:	learn: 336.2738263	total: 175ms	remaining: 15.8s
11:	learn: 327.2901027	total: 176ms	remaining: 14.5s
12:	learn: 317.9919302	total: 177ms	remaining: 13.4s
13:	learn: 309.9682014	total: 178ms	remaining: 12.5s
14:	learn: 301.4406142	total: 179ms	remaining: 11.7s
15:	learn: 293.0847208	total: 179ms	remaining: 11s
16:	learn: 285.3581312	total: 180ms	remaining: 10.4s
17:	learn: 277.6905120	total: 181ms	remaining: 9.86s
18:	learn: 270.2580731	tot

258:	learn: 24.7719877	total: 351ms	remaining: 1s
259:	learn: 24.6277509	total: 351ms	remaining: 1000ms
260:	learn: 24.6153634	total: 352ms	remaining: 997ms
261:	learn: 24.5229142	total: 353ms	remaining: 994ms
262:	learn: 24.4392167	total: 354ms	remaining: 991ms
263:	learn: 24.4247542	total: 354ms	remaining: 988ms
264:	learn: 24.4055225	total: 355ms	remaining: 985ms
265:	learn: 24.2917856	total: 356ms	remaining: 981ms
266:	learn: 24.1473693	total: 356ms	remaining: 978ms
267:	learn: 24.1286369	total: 357ms	remaining: 975ms
268:	learn: 24.0974606	total: 358ms	remaining: 973ms
269:	learn: 23.9688100	total: 359ms	remaining: 970ms
270:	learn: 23.9530801	total: 359ms	remaining: 967ms
271:	learn: 23.8780432	total: 360ms	remaining: 964ms
272:	learn: 23.8556067	total: 361ms	remaining: 960ms
273:	learn: 23.7755287	total: 361ms	remaining: 957ms
274:	learn: 23.7668576	total: 362ms	remaining: 954ms
275:	learn: 23.7586976	total: 363ms	remaining: 952ms
276:	learn: 23.6596378	total: 363ms	remaining: 9

488:	learn: 15.2803451	total: 512ms	remaining: 535ms
489:	learn: 15.2523881	total: 513ms	remaining: 533ms
490:	learn: 15.2393104	total: 513ms	remaining: 532ms
491:	learn: 15.1943009	total: 514ms	remaining: 531ms
492:	learn: 15.1659591	total: 515ms	remaining: 529ms
493:	learn: 15.1270935	total: 515ms	remaining: 528ms
494:	learn: 15.0880920	total: 516ms	remaining: 526ms
495:	learn: 15.0655073	total: 517ms	remaining: 525ms
496:	learn: 15.0406381	total: 517ms	remaining: 524ms
497:	learn: 15.0141564	total: 518ms	remaining: 522ms
498:	learn: 14.9892279	total: 519ms	remaining: 521ms
499:	learn: 14.9735867	total: 520ms	remaining: 520ms
500:	learn: 14.9289371	total: 520ms	remaining: 518ms
501:	learn: 14.9071030	total: 521ms	remaining: 517ms
502:	learn: 14.8824394	total: 522ms	remaining: 515ms
503:	learn: 14.8698834	total: 522ms	remaining: 514ms
504:	learn: 14.8613601	total: 523ms	remaining: 513ms
505:	learn: 14.8364310	total: 524ms	remaining: 511ms
506:	learn: 14.7965683	total: 525ms	remaining:

750:	learn: 10.2101401	total: 693ms	remaining: 230ms
751:	learn: 10.2036717	total: 694ms	remaining: 229ms
752:	learn: 10.1777488	total: 695ms	remaining: 228ms
753:	learn: 10.1649462	total: 695ms	remaining: 227ms
754:	learn: 10.1545791	total: 696ms	remaining: 226ms
755:	learn: 10.1328069	total: 697ms	remaining: 225ms
756:	learn: 10.1173152	total: 698ms	remaining: 224ms
757:	learn: 10.1031453	total: 698ms	remaining: 223ms
758:	learn: 10.0982894	total: 699ms	remaining: 222ms
759:	learn: 10.0952482	total: 700ms	remaining: 221ms
760:	learn: 10.0719582	total: 700ms	remaining: 220ms
761:	learn: 10.0555908	total: 701ms	remaining: 219ms
762:	learn: 10.0491279	total: 702ms	remaining: 218ms
763:	learn: 10.0430631	total: 702ms	remaining: 217ms
764:	learn: 10.0382810	total: 703ms	remaining: 216ms
765:	learn: 10.0196294	total: 704ms	remaining: 215ms
766:	learn: 9.9951746	total: 704ms	remaining: 214ms
767:	learn: 9.9726881	total: 705ms	remaining: 213ms
768:	learn: 9.9658643	total: 706ms	remaining: 21

Learning rate set to 0.032162
0:	learn: 445.2529805	total: 1.05ms	remaining: 1.05s
1:	learn: 434.0237971	total: 2.1ms	remaining: 1.05s
2:	learn: 422.4162535	total: 3.04ms	remaining: 1.01s
3:	learn: 411.8167591	total: 3.9ms	remaining: 971ms
4:	learn: 401.6562116	total: 4.72ms	remaining: 939ms
5:	learn: 391.7220467	total: 5.54ms	remaining: 919ms
6:	learn: 381.8107586	total: 6.38ms	remaining: 905ms
7:	learn: 372.1632874	total: 7.19ms	remaining: 892ms
8:	learn: 362.6829888	total: 7.89ms	remaining: 868ms
9:	learn: 353.1758903	total: 8.62ms	remaining: 854ms
10:	learn: 344.2376907	total: 9.33ms	remaining: 839ms
11:	learn: 335.2054615	total: 10ms	remaining: 827ms
12:	learn: 326.4793574	total: 10.8ms	remaining: 816ms
13:	learn: 318.5255016	total: 11.4ms	remaining: 806ms
14:	learn: 310.6297892	total: 12.1ms	remaining: 795ms
15:	learn: 302.8359970	total: 12.7ms	remaining: 781ms
16:	learn: 295.3534064	total: 13.3ms	remaining: 770ms
17:	learn: 287.5305505	total: 13.9ms	remaining: 759ms
18:	learn: 2

...

871:	learn: 7.4742563	total: 554ms	remaining: 81.3ms
872:	learn: 7.4607121	total: 554ms	remaining: 80.6ms
873:	learn: 7.4585998	total: 555ms	remaining: 80ms
874:	learn: 7.4451470	total: 556ms	remaining: 79.4ms
875:	learn: 7.4328762	total: 556ms	remaining: 78.7ms
876:	learn: 7.4178108	total: 557ms	remaining: 78.1ms
877:	learn: 7.4101928	total: 557ms	remaining: 77.4ms
878:	learn: 7.4051651	total: 558ms	remaining: 76.8ms
879:	learn: 7.3978472	total: 559ms	remaining: 76.2ms
880:	learn: 7.3853069	total: 559ms	remaining: 75.5ms
881:	learn: 7.3757139	total: 560ms	remaining: 74.9ms
882:	learn: 7.3637375	total: 560ms	remaining: 74.3ms
883:	learn: 7.3571937	total: 561ms	remaining: 73.6ms
884:	learn: 7.3485776	total: 562ms	remaining: 73ms
885:	learn: 7.3414508	total: 562ms	remaining: 72.4ms
886:	learn: 7.3328397	total: 563ms	remaining: 71.7ms
887:	learn: 7.3204313	total: 564ms	remaining: 71.1ms
888:	learn: 7.2994663	total: 564ms	remaining: 70.5ms
889:	learn: 7.2852682	total: 565ms	remaining: 69.8

74:	learn: 84.4789071	total: 48.9ms	remaining: 603ms
75:	learn: 82.8844574	total: 49.6ms	remaining: 603ms
76:	learn: 81.5515874	total: 50.3ms	remaining: 604ms
77:	learn: 80.0529800	total: 51ms	remaining: 603ms
78:	learn: 78.7611922	total: 51.6ms	remaining: 602ms
79:	learn: 77.4882751	total: 52.2ms	remaining: 601ms
80:	learn: 76.2319193	total: 52.8ms	remaining: 600ms
81:	learn: 74.8078183	total: 53.5ms	remaining: 599ms
82:	learn: 73.5622478	total: 54.2ms	remaining: 598ms
83:	learn: 72.4924318	total: 54.9ms	remaining: 599ms
84:	learn: 71.3833821	total: 55.5ms	remaining: 598ms
85:	learn: 70.4351553	total: 56.2ms	remaining: 597ms
86:	learn: 69.4329726	total: 56.8ms	remaining: 596ms
87:	learn: 68.4179256	total: 57.5ms	remaining: 595ms
88:	learn: 67.4789432	total: 58.1ms	remaining: 595ms
89:	learn: 66.5444346	total: 58.7ms	remaining: 594ms
90:	learn: 65.6446898	total: 59.4ms	remaining: 593ms
91:	learn: 64.7004127	total: 60ms	remaining: 592ms
92:	learn: 63.8686650	total: 60.6ms	remaining: 591

358:	learn: 15.5600630	total: 221ms	remaining: 394ms
359:	learn: 15.4953921	total: 221ms	remaining: 394ms
360:	learn: 15.4748293	total: 222ms	remaining: 393ms
361:	learn: 15.4619892	total: 223ms	remaining: 392ms
362:	learn: 15.4135173	total: 223ms	remaining: 392ms
363:	learn: 15.3930082	total: 224ms	remaining: 391ms
364:	learn: 15.3804869	total: 224ms	remaining: 391ms
365:	learn: 15.3682518	total: 225ms	remaining: 390ms
366:	learn: 15.3286621	total: 226ms	remaining: 389ms
367:	learn: 15.2831571	total: 226ms	remaining: 389ms
368:	learn: 15.2409382	total: 227ms	remaining: 388ms
369:	learn: 15.2350472	total: 228ms	remaining: 387ms
370:	learn: 15.1796929	total: 228ms	remaining: 387ms
371:	learn: 15.1101002	total: 229ms	remaining: 386ms
372:	learn: 15.0909932	total: 230ms	remaining: 386ms
373:	learn: 15.0714343	total: 230ms	remaining: 385ms
374:	learn: 15.0654655	total: 231ms	remaining: 385ms
375:	learn: 15.0530383	total: 231ms	remaining: 384ms
376:	learn: 15.0321074	total: 232ms	remaining:

663:	learn: 8.1452771	total: 406ms	remaining: 205ms
664:	learn: 8.1243854	total: 407ms	remaining: 205ms
665:	learn: 8.1176960	total: 407ms	remaining: 204ms
666:	learn: 8.1000243	total: 408ms	remaining: 204ms
667:	learn: 8.0750825	total: 409ms	remaining: 203ms
668:	learn: 8.0656474	total: 409ms	remaining: 202ms
669:	learn: 8.0484392	total: 410ms	remaining: 202ms
670:	learn: 8.0420513	total: 410ms	remaining: 201ms
671:	learn: 8.0389083	total: 411ms	remaining: 201ms
672:	learn: 8.0107335	total: 412ms	remaining: 200ms
673:	learn: 8.0007084	total: 412ms	remaining: 199ms
674:	learn: 7.9796744	total: 413ms	remaining: 199ms
675:	learn: 7.9759689	total: 414ms	remaining: 198ms
676:	learn: 7.9697855	total: 414ms	remaining: 198ms
677:	learn: 7.9565382	total: 415ms	remaining: 197ms
678:	learn: 7.9489571	total: 415ms	remaining: 196ms
679:	learn: 7.9415879	total: 416ms	remaining: 196ms
680:	learn: 7.9252459	total: 417ms	remaining: 195ms
681:	learn: 7.8983718	total: 417ms	remaining: 195ms
682:	learn: 

939:	learn: 5.4442145	total: 574ms	remaining: 36.6ms
940:	learn: 5.4423882	total: 574ms	remaining: 36ms
941:	learn: 5.4372735	total: 575ms	remaining: 35.4ms
942:	learn: 5.4295634	total: 576ms	remaining: 34.8ms
943:	learn: 5.4202948	total: 576ms	remaining: 34.2ms
944:	learn: 5.4181898	total: 577ms	remaining: 33.6ms
945:	learn: 5.4076760	total: 578ms	remaining: 33ms
946:	learn: 5.3972628	total: 578ms	remaining: 32.4ms
947:	learn: 5.3916377	total: 579ms	remaining: 31.8ms
948:	learn: 5.3850651	total: 580ms	remaining: 31.1ms
949:	learn: 5.3817532	total: 580ms	remaining: 30.5ms
950:	learn: 5.3790206	total: 581ms	remaining: 29.9ms
951:	learn: 5.3741027	total: 581ms	remaining: 29.3ms
952:	learn: 5.3696382	total: 582ms	remaining: 28.7ms
953:	learn: 5.3649971	total: 583ms	remaining: 28.1ms
954:	learn: 5.3560843	total: 583ms	remaining: 27.5ms
955:	learn: 5.3509412	total: 584ms	remaining: 26.9ms
956:	learn: 5.3422787	total: 585ms	remaining: 26.3ms
957:	learn: 5.3312373	total: 586ms	remaining: 25.7

In [15]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.99060316 0.99520303 0.99373591]


In [16]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.993 +/- 0.002


In [17]:
CatB = CatBoostRegressor()
Y = Y_train.astype('int32')
CatB = CatB.fit(X_train,Y)
scores = cross_val_score(CatB, X=X_train, y=Y, cv=5, n_jobs=1)

Learning rate set to 0.034298
0:	learn: 445.3383297	total: 1.12ms	remaining: 1.12s
1:	learn: 433.3776846	total: 2.36ms	remaining: 1.18s
2:	learn: 420.9934949	total: 3.35ms	remaining: 1.11s
3:	learn: 408.9589394	total: 4.18ms	remaining: 1.04s
4:	learn: 398.3737284	total: 4.92ms	remaining: 978ms
5:	learn: 387.0923653	total: 5.72ms	remaining: 949ms
6:	learn: 376.2407186	total: 6.5ms	remaining: 922ms
7:	learn: 365.7209279	total: 7.46ms	remaining: 925ms
8:	learn: 355.4582600	total: 8.21ms	remaining: 904ms
9:	learn: 345.7560879	total: 8.94ms	remaining: 885ms
10:	learn: 336.2738263	total: 9.65ms	remaining: 867ms
11:	learn: 327.2901027	total: 10.3ms	remaining: 851ms
12:	learn: 317.9919302	total: 11ms	remaining: 838ms
13:	learn: 309.9682014	total: 11.8ms	remaining: 829ms
14:	learn: 301.4406142	total: 12.4ms	remaining: 817ms
15:	learn: 293.0847208	total: 13.1ms	remaining: 808ms
16:	learn: 285.3581312	total: 14ms	remaining: 810ms
17:	learn: 277.6905120	total: 14.7ms	remaining: 803ms
18:	learn: 27

241:	learn: 25.5907584	total: 178ms	remaining: 557ms
242:	learn: 25.5773751	total: 179ms	remaining: 557ms
243:	learn: 25.5629054	total: 180ms	remaining: 556ms
244:	learn: 25.5340643	total: 180ms	remaining: 556ms
245:	learn: 25.4768569	total: 181ms	remaining: 555ms
246:	learn: 25.4256098	total: 182ms	remaining: 555ms
247:	learn: 25.3468756	total: 183ms	remaining: 554ms
248:	learn: 25.2930487	total: 184ms	remaining: 554ms
249:	learn: 25.2810419	total: 184ms	remaining: 553ms
250:	learn: 25.1757119	total: 185ms	remaining: 552ms
251:	learn: 25.1463752	total: 186ms	remaining: 552ms
252:	learn: 25.0872729	total: 187ms	remaining: 551ms
253:	learn: 25.0718468	total: 187ms	remaining: 550ms
254:	learn: 24.9943436	total: 188ms	remaining: 551ms
255:	learn: 24.9774379	total: 189ms	remaining: 550ms
256:	learn: 24.9364243	total: 190ms	remaining: 550ms
257:	learn: 24.9208038	total: 191ms	remaining: 550ms
258:	learn: 24.7719877	total: 192ms	remaining: 549ms
259:	learn: 24.6277509	total: 193ms	remaining:

467:	learn: 15.9015805	total: 344ms	remaining: 391ms
468:	learn: 15.8805092	total: 345ms	remaining: 390ms
469:	learn: 15.8681828	total: 346ms	remaining: 390ms
470:	learn: 15.8508635	total: 346ms	remaining: 389ms
471:	learn: 15.8294520	total: 347ms	remaining: 388ms
472:	learn: 15.8033812	total: 348ms	remaining: 388ms
473:	learn: 15.7587557	total: 349ms	remaining: 387ms
474:	learn: 15.7544721	total: 349ms	remaining: 386ms
475:	learn: 15.7323620	total: 350ms	remaining: 385ms
476:	learn: 15.7205713	total: 351ms	remaining: 385ms
477:	learn: 15.6837740	total: 352ms	remaining: 384ms
478:	learn: 15.6682829	total: 352ms	remaining: 383ms
479:	learn: 15.6271646	total: 353ms	remaining: 382ms
480:	learn: 15.6004012	total: 354ms	remaining: 382ms
481:	learn: 15.5654769	total: 354ms	remaining: 381ms
482:	learn: 15.5068131	total: 355ms	remaining: 380ms
483:	learn: 15.4804127	total: 356ms	remaining: 379ms
484:	learn: 15.4536549	total: 357ms	remaining: 379ms
485:	learn: 15.4042363	total: 357ms	remaining:

716:	learn: 10.6442363	total: 519ms	remaining: 205ms
717:	learn: 10.6341589	total: 520ms	remaining: 204ms
718:	learn: 10.6134201	total: 521ms	remaining: 204ms
719:	learn: 10.6072615	total: 522ms	remaining: 203ms
720:	learn: 10.5984237	total: 522ms	remaining: 202ms
721:	learn: 10.5767993	total: 523ms	remaining: 201ms
722:	learn: 10.5658267	total: 524ms	remaining: 201ms
723:	learn: 10.5371679	total: 525ms	remaining: 200ms
724:	learn: 10.5206926	total: 525ms	remaining: 199ms
725:	learn: 10.5041126	total: 526ms	remaining: 199ms
726:	learn: 10.4997615	total: 527ms	remaining: 198ms
727:	learn: 10.4906676	total: 527ms	remaining: 197ms
728:	learn: 10.4677917	total: 528ms	remaining: 196ms
729:	learn: 10.4606702	total: 529ms	remaining: 196ms
730:	learn: 10.4581429	total: 530ms	remaining: 195ms
731:	learn: 10.4479357	total: 530ms	remaining: 194ms
732:	learn: 10.4432759	total: 531ms	remaining: 193ms
733:	learn: 10.4309462	total: 532ms	remaining: 193ms
734:	learn: 10.4278571	total: 532ms	remaining:

954:	learn: 8.0247371	total: 684ms	remaining: 32.2ms
955:	learn: 8.0093856	total: 685ms	remaining: 31.5ms
956:	learn: 7.9938841	total: 686ms	remaining: 30.8ms
957:	learn: 7.9886380	total: 687ms	remaining: 30.1ms
958:	learn: 7.9696640	total: 687ms	remaining: 29.4ms
959:	learn: 7.9678819	total: 688ms	remaining: 28.7ms
960:	learn: 7.9570875	total: 689ms	remaining: 27.9ms
961:	learn: 7.9529750	total: 689ms	remaining: 27.2ms
962:	learn: 7.9474723	total: 690ms	remaining: 26.5ms
963:	learn: 7.9358957	total: 691ms	remaining: 25.8ms
964:	learn: 7.9204168	total: 691ms	remaining: 25.1ms
965:	learn: 7.9189949	total: 692ms	remaining: 24.4ms
966:	learn: 7.9080314	total: 693ms	remaining: 23.6ms
967:	learn: 7.8981445	total: 694ms	remaining: 22.9ms
968:	learn: 7.8873172	total: 694ms	remaining: 22.2ms
969:	learn: 7.8859908	total: 695ms	remaining: 21.5ms
970:	learn: 7.8753721	total: 696ms	remaining: 20.8ms
971:	learn: 7.8600367	total: 697ms	remaining: 20.1ms
972:	learn: 7.8516893	total: 698ms	remaining: 

...

Learning rate set to 0.033114
0:	learn: 442.7118658	total: 858us	remaining: 858ms
1:	learn: 431.7946257	total: 1.61ms	remaining: 802ms
2:	learn: 420.1358389	total: 2.33ms	remaining: 773ms
3:	learn: 409.4462795	total: 3.02ms	remaining: 752ms
4:	learn: 399.4282141	total: 3.7ms	remaining: 736ms
5:	learn: 388.8437670	total: 4.42ms	remaining: 732ms
6:	learn: 378.6382081	total: 5.13ms	remaining: 728ms
7:	learn: 368.5558222	total: 5.82ms	remaining: 722ms
8:	learn: 358.8837218	total: 6.57ms	remaining: 724ms
9:	learn: 349.5832184	total: 7.34ms	remaining: 727ms
10:	learn: 340.8434407	total: 8.13ms	remaining: 731ms
11:	learn: 332.1731344	total: 8.9ms	remaining: 733ms
12:	learn: 323.3115713	total: 9.6ms	remaining: 729ms
13:	learn: 315.3186255	total: 10.4ms	remaining: 733ms
14:	learn: 307.5203936	total: 11.2ms	remaining: 735ms
15:	learn: 299.5182639	total: 11.9ms	remaining: 732ms
16:	learn: 291.9409975	total: 12.6ms	remaining: 730ms
17:	learn: 284.5745423	total: 13.3ms	remaining: 725ms
18:	learn: 2

245:	learn: 24.2023870	total: 168ms	remaining: 514ms
246:	learn: 24.1343017	total: 169ms	remaining: 514ms
247:	learn: 24.1077440	total: 169ms	remaining: 514ms
248:	learn: 24.0690852	total: 170ms	remaining: 513ms
249:	learn: 24.0533423	total: 171ms	remaining: 513ms
250:	learn: 23.9589228	total: 172ms	remaining: 512ms
251:	learn: 23.9371321	total: 172ms	remaining: 511ms
252:	learn: 23.8559249	total: 173ms	remaining: 511ms
253:	learn: 23.8280917	total: 174ms	remaining: 510ms
254:	learn: 23.7948002	total: 174ms	remaining: 509ms
255:	learn: 23.7016824	total: 175ms	remaining: 509ms
256:	learn: 23.6572177	total: 176ms	remaining: 508ms
257:	learn: 23.6001277	total: 177ms	remaining: 508ms
258:	learn: 23.5517706	total: 177ms	remaining: 507ms
259:	learn: 23.5382201	total: 178ms	remaining: 507ms
260:	learn: 23.5266966	total: 179ms	remaining: 506ms
261:	learn: 23.4908164	total: 180ms	remaining: 506ms
262:	learn: 23.4383031	total: 180ms	remaining: 505ms
263:	learn: 23.4178986	total: 181ms	remaining:

486:	learn: 14.0851804	total: 332ms	remaining: 350ms
487:	learn: 14.0519831	total: 333ms	remaining: 349ms
488:	learn: 14.0077931	total: 334ms	remaining: 349ms
489:	learn: 13.9717468	total: 335ms	remaining: 348ms
490:	learn: 13.9490859	total: 335ms	remaining: 348ms
491:	learn: 13.9173546	total: 336ms	remaining: 347ms
492:	learn: 13.8918760	total: 337ms	remaining: 346ms
493:	learn: 13.8479190	total: 337ms	remaining: 346ms
494:	learn: 13.8233478	total: 338ms	remaining: 345ms
495:	learn: 13.7867983	total: 339ms	remaining: 344ms
496:	learn: 13.7530832	total: 339ms	remaining: 344ms
497:	learn: 13.7119404	total: 340ms	remaining: 343ms
498:	learn: 13.6816010	total: 341ms	remaining: 342ms
499:	learn: 13.6713531	total: 342ms	remaining: 342ms
500:	learn: 13.6495077	total: 343ms	remaining: 341ms
501:	learn: 13.6161866	total: 344ms	remaining: 341ms
502:	learn: 13.5808783	total: 344ms	remaining: 340ms
503:	learn: 13.5403554	total: 345ms	remaining: 340ms
504:	learn: 13.4978582	total: 346ms	remaining:

740:	learn: 9.2162535	total: 507ms	remaining: 177ms
741:	learn: 9.1969157	total: 507ms	remaining: 176ms
742:	learn: 9.1793747	total: 508ms	remaining: 176ms
743:	learn: 9.1649494	total: 509ms	remaining: 175ms
744:	learn: 9.1532362	total: 510ms	remaining: 174ms
745:	learn: 9.1340947	total: 510ms	remaining: 174ms
746:	learn: 9.1290529	total: 511ms	remaining: 173ms
747:	learn: 9.1247240	total: 512ms	remaining: 172ms
748:	learn: 9.1013219	total: 512ms	remaining: 172ms
749:	learn: 9.0895588	total: 513ms	remaining: 171ms
750:	learn: 9.0798562	total: 514ms	remaining: 170ms
751:	learn: 9.0659953	total: 514ms	remaining: 170ms
752:	learn: 9.0566241	total: 515ms	remaining: 169ms
753:	learn: 9.0383225	total: 516ms	remaining: 168ms
754:	learn: 9.0309400	total: 516ms	remaining: 168ms
755:	learn: 9.0203054	total: 517ms	remaining: 167ms
756:	learn: 9.0112619	total: 518ms	remaining: 166ms
757:	learn: 9.0039886	total: 519ms	remaining: 166ms
758:	learn: 8.9940017	total: 519ms	remaining: 165ms
759:	learn: 

991:	learn: 6.4602037	total: 678ms	remaining: 5.47ms
992:	learn: 6.4509089	total: 679ms	remaining: 4.79ms
993:	learn: 6.4371018	total: 680ms	remaining: 4.1ms
994:	learn: 6.4283691	total: 680ms	remaining: 3.42ms
995:	learn: 6.4247762	total: 681ms	remaining: 2.73ms
996:	learn: 6.4126202	total: 682ms	remaining: 2.05ms
997:	learn: 6.4101704	total: 683ms	remaining: 1.37ms
998:	learn: 6.4042746	total: 683ms	remaining: 683us
999:	learn: 6.3973309	total: 684ms	remaining: 0us


In [18]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.98886728 0.99468376 0.99333479 0.99564036 0.99635888]


In [19]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.994 +/- 0.003


# XGBoost

In [20]:
XGB = xgb.XGBRegressor()
XGB = XGB.fit(X_train,Y_train)
scores = cross_val_score(XGB, X=X_train, y=Y, cv=3, n_jobs=1)

In [21]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.99036433 0.98016549 0.98878904]


In [22]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.986 +/- 0.004


In [23]:
XGB = xgb.XGBRegressor()
XGB = XGB.fit(X_train,Y_train)
scores = cross_val_score(XGB, X=X_train, y=Y, cv=5, n_jobs=1)

In [24]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.98612182 0.99260054 0.98616239 0.99142316 0.98697334]


In [25]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.989 +/- 0.003


# LightGBM 

In [26]:
LGBM = lgb.LGBMRegressor()
Y = Y_train.astype('int32')
LGBM = LGBM.fit(X_train,Y)
scores = cross_val_score(LGBM, X=X_train, y=Y, cv=3, n_jobs=1)

In [27]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.96598097 0.97695088 0.9853255 ]


In [28]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.976 +/- 0.008


In [30]:
LGBM = lgb.LGBMRegressor()
Y = Y_train.astype('int32')
LGBM = LGBM.fit(X_train,Y)
scores = cross_val_score(LGBM, X=X_train, y=Y, cv=5, n_jobs=1)

In [31]:
print('CV accuracy scores: %s' % scores)

CV accuracy scores: [0.98306544 0.98228726 0.9861191  0.9907518  0.98895344]


In [32]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

CV accuracy: 0.986 +/- 0.003
